# week 3
## goals:
- build boxes around the MPAs
- try to see which had activity pre-closure

### largest MPAs:
1. Marae Moana (Cook Islands) [-165, -155]lat, [-25, -10]lng
2. Ross Sea Region (Ross Sea)
3. Papahanaumokuakea Marine National Monument (PMNM) [177, -160], [19, 32]
4. Natural Park of the Coral Sea
5. Pacific Remote Islands (expanded)
6. Coral Sea Marine Park
7. French Southern Territories Marine Reserve
8. Phoenix Islands
9. Stellar Sea Lion Protection Areas (just Aleutian Islands 2015)


In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import os

In [2]:
class Point():
    def __init__(self, mmsi, lat, long, fishing_hours, date):
        self.mmsi = mmsi
        self.lat = lat/100.0
        self.long = long/100.0
        self.fishing_hours = fishing_hours
        self.date = datetime.fromisoformat(date)
    
    def __str__(self):
        return f"mmsi:{self.mmsi} lat:{self.lat} long:{self.long} fishing hours:{self.fishing_hours} date:{self.date}"

class MPA():
    def __init__(self, name, lat_lower, lat_upper, long_lower, long_upper, datestring):
        self.name = name
        self.points = []
        self.lat_lower = lat_lower
        self.lat_upper = lat_upper
        self.long_lower = long_lower
        self.long_upper = long_upper
        
        self.date = datetime.fromisoformat(datestring)
    
    def contains(self, point):
        if point.date > self.date:
            return False
        
        if not (point.long >= self.long_lower and point.long <= self.long_upper):
            return False
        
        # possible if we're at the antimeridian
        lat_reversed = self.lat_lower > self.lat_upper
        if lat_reversed:
            return (point.lat <= self.lat_lower and point.lat >= self.lat_upper)
        else:
            return (point.lat <= self.lat_upper and point.lat <= self.lat_upper)
            
    
    def add_point(self, point):
        if self.contains(point):
            self.points.append(point)
        


In [3]:
## Load all the data (this is kinda slow)

MMSI = 3
LAT = 1
LONG = 2
DATE = 0
FISHING_HOURS = 4

points = []
def load_file(filename):
    r = pd.read_csv(filename).to_numpy()
    for row in r:
        points.append(Point(row[MMSI], row[LAT], row[LONG], row[FISHING_HOURS], row[DATE]))

filenames = os.listdir('data/daily_csvs')
counted = 0
for filename in filenames:
    print(f'\r {filename} {counted}/{len(filenames)}', end='')
    load_file('data/daily_csvs/' + filename)
    counted += 1

 2013-05-12.csv 1616/1827

KeyboardInterrupt: 

In [ ]:
## Add to the MPAs:

mpas = [
    MPA("Marae Moana (Cook Islands)", -165, -155, -25, -10, "2017-07-13"),
    MPA("Papahanaumokuakea", 177, -160, 19, 32, "2016-08-01"),
    MPA("Ross Sea", 150, -150, -80, -67, "2017-12-01")
]

for point in points:
    for mpa in mpas:
        mpa.add_point(point)


In [ ]:
for mpa in mpas:
    
    mpa.hours_by_mmsi = {}
    for p in mpa.points:
        if p.mmsi not in mpa.hours_by_mmsi:
            mpa.hours_by_mmsi[p.mmsi] = 0
        mpa.hours_by_mmsi[p.mmsi] += p.fishing_hours

In [ ]:
for mpa in mpas:
    print(mpa.name)
    print(len(mpa.hours_by_mmsi))
    hours = sum([p.fishing_hours for p in mpa.points])
    print(hours)
    print(hours / (365 * 4))